In [1]:
from sqlalchemy.orm import DeclarativeBase # Декларативный, базовый класс
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import Session # для создания записей в бд
from sqlalchemy.orm import Mapped
from sqlalchemy import ForeignKey, String
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine, text, select
from typing import List
import sqlite3

In [3]:
# объявляем модели

class Base(DeclarativeBase):
  pass


class User(Base):
  __tablename__ = "users"

  id: Mapped[int] = mapped_column(primary_key=True)
  name: Mapped[str] = mapped_column(String(30))
  username: Mapped[str | None]
  addresses: Mapped[list['Address']] = relationship( # Если мы указали List, значит у одного адреса, может быть множество юзеров
      back_populates = "user", # здесь связь 1 ко многим
      cascade = "all, delete-orphan" # что делаем когда удаляем user'а (all - удаление всех записей)
  )

class Address(Base):
  __tablename__ = "addresses"

  id: Mapped[int] = mapped_column(primary_key=True)
  email: Mapped[str]
  user_id: Mapped[int] = mapped_column(ForeignKey("users.id"))
  user: Mapped["User"] = relationship(back_populates="addresses") # настройка связи 1 ко многим

In [4]:
# Создаем модели в базе данных
engine = create_engine(
    url="sqlite:///./db.sqlite3"
)
Base.metadata.create_all(bind=engine)

In [5]:
with Session(engine) as session:

  user = User(
      name="Bob Grow",
      username="Bom",
      addresses = [
          Address(
              email="bob@mail.ru"
          ),
          Address(
              email="bob_two@mail.ru"
          )
      ]
  )
  session.add(user)
  session.commit()

In [13]:
stmp = select(User).where(User.name=="Bob Grow")
user: User | None = session.scalar(stmp)
user